## HW1_Harshini_Tammareddy

### Dependencies

In [1]:

##Loading required packages
from dvc.api import read,get_url
import pandas as pd
import string
import re

##packages needed for text cleaning
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stopword = set(stopwords.words('english')) #stopwords
from textblob import TextBlob## sentiment analysis
##misc
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VIOLIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VIOLIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VIOLIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading data

In [2]:
##data loading
txt = read('resources/data/shakespeare/shakespeare.txt', 
           repo='https://github.com/TLP-COI/text-data-course')

print(txt[:250]) ##output

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



#### Assumptions about shape of text - 

- The text has only two aspects. Dialogues and speakers of the dialogues. 
- Each speaker is followed by his dialogue. 
- The dialogues are not continuous and randomly assigned one after the other. 

In [3]:
##seperating speaker words
speaker = re.findall( '[A-Z]\w+\s\w+:|[A-Z]\w+:',txt)
speaker

['First Citizen:',
 'All:',
 'First Citizen:',
 'All:',
 'First Citizen:',
 'All:',
 'First Citizen:',
 'All:',
 'Second Citizen:',
 'First Citizen:',
 'Second Citizen:',
 'All:',
 'Second Citizen:',
 'First Citizen:',
 'Second Citizen:',
 'First Citizen:',
 'Second Citizen:',
 'First Citizen:',
 'All:',
 'First Citizen:',
 'Second Citizen:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'First Citizen:',
 'MENENIUS:',
 'MARCIUS:',
 'First Citizen:',
 'MARCIUS:',
 'MENENIUS:',
 'MARCIUS:',
 'MENENIUS:',
 'MARCIUS:',
 'MENENIUS:',
 'MARCIUS:',
 'MENENIUS:',
 'MARCIUS:',
 'Messenger:',
 'MARCIUS:',
 'Here:',
 'Messenger:',
 'MARCIUS:',
 'First

In [4]:
##converting into dataframe
speaker_df = pd.DataFrame(speaker, columns=['Speaker'])
print(speaker_df) ##output

             Speaker
0     First Citizen:
1               All:
2     First Citizen:
3               All:
4     First Citizen:
...              ...
7440        No more:
7441      SEBASTIAN:
7442        ANTONIO:
7443      SEBASTIAN:
7444        ANTONIO:

[7445 rows x 1 columns]


In [5]:
##cleaning the output
speaker_df = speaker_df.replace(r':',' ', regex=True) 
 ##output



Assumptions about Speaker 
- Speaker has either one or max two words.  
- Speaker word always starts with capital letter
- Speaker word always ends with Colon - :


In [6]:
dialogue = re.split('[A-Z]\w+\s\w+:|[A-Z]\w+:',txt) ##filtering the dialogue. 
dialogue ## output

['',
 '\nBefore we proceed any further, hear me speak.\n\n',
 '\nSpeak, speak.\n\n',
 '\nYou are all resolved rather to die than to famish?\n\n',
 '\nResolved. resolved.\n\n',
 '\nFirst, you know Caius Marcius is chief enemy to the people.\n\n',
 "\nWe know't, we know't.\n\n",
 "\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\n",
 "\nNo more talking on't; let it be done: away, away!\n\n",
 '\nOne word, good citizens.\n\n',
 '\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger for bread, not in thirst for revenge.\n\n',
 '\nWo

In [7]:
dialogue_df = pd.DataFrame(dialogue, columns=['Dialogue']) ##converting the list into dataframe
print(dialogue_df)

                                               Dialogue
0                                                      
1     \nBefore we proceed any further, hear me speak...
2                                   \nSpeak, speak.\n\n
3     \nYou are all resolved rather to die than to f...
4                             \nResolved. resolved.\n\n
...                                                 ...
7441  --\nAnd yet me thinks I see it in thy face,\nW...
7442                       \nWhat, art thou waking?\n\n
7443                    \nDo you not hear me speak?\n\n
7444  \nI do; and surely\nIt is a sleepy language an...
7445  \nNoble Sebastian,\nThou let'st thy fortune sl...

[7446 rows x 1 columns]


#### Assumptions - Dialogue 
- Each dialogue is sandwitched between \n. 
- New Dialogue starts after the name of the speaker.
- The text has just speaker and dialogues and hence once the speakers data is dropped, the remaining text is set of dialogues seperated by \n.

In [8]:
dialogue_df.head(10) ##output df

,Dialogue
0,
1,"\nBefore we proceed any further, hear me speak..."
2,"\nSpeak, speak.\n\n"
3,\nYou are all resolved rather to die than to f...
4,\nResolved. resolved.\n\n
5,"\nFirst, you know Caius Marcius is chief enemy..."
6,"\nWe know't, we know't.\n\n"
7,"\nLet us kill him, and we'll have corn at our ..."
8,"\nNo more talking on't; let it be done: away, ..."
9,"\nOne word, good citizens.\n\n"


In [9]:
dialogue_df = dialogue_df.replace(r'\n',' ', regex=True)  ##Cleaning the dataframe

In [10]:
dialogue_df #output

,Dialogue
0,
1,"Before we proceed any further, hear me speak."
2,"Speak, speak."
3,You are all resolved rather to die than to fa...
4,Resolved. resolved.
...,...
7441,"-- And yet me thinks I see it in thy face, Wha..."
7442,"What, art thou waking?"
7443,Do you not hear me speak?
7444,I do; and surely It is a sleepy language and ...


In [11]:
result = pd.concat([speaker_df, dialogue_df], axis=1)  ##
result

,Speaker,Dialogue
0,First Citizen,
1,All,"Before we proceed any further, hear me speak."
2,First Citizen,"Speak, speak."
3,All,You are all resolved rather to die than to fa...
4,First Citizen,Resolved. resolved.
...,...,...
7441,SEBASTIAN,"-- And yet me thinks I see it in thy face, Wha..."
7442,ANTONIO,"What, art thou waking?"
7443,SEBASTIAN,Do you not hear me speak?
7444,ANTONIO,I do; and surely It is a sleepy language and ...


In [12]:
result.dtypes

Speaker     object
Dialogue    object
dtype: object

In [13]:
result['Dialogue'] = result['Dialogue'].astype('str')
result['Speaker'] = result['Speaker'].astype('str')
result['Speaker'] = result['Speaker'].str.lower()

### Part 2

A new dataset with names of the characters in Shakespear's books along with the name of the 
play was added to figure out the play name of each dialogue. The speakers columns is compared with the character's column after bringing uniformity in the both the speaker and character columns. 

Assumptions - 

- Speakers are all characters in the shakespear's play. 
- the names of speakers and characters(from the new dataset) are uniform. 

In [14]:
##loading data
data1 =  pd.read_csv("play.csv")

In [15]:
## cleaning this data

In [16]:
data1["Character"] = data1["Character"].str.lower()
data1["Play"] = data1["Play"].str.lower()
data1["Speaker"] = data1["Character"]
del data1["Character"]
data1.head(3)

,Play,Speaker
0,(the merchant of venice),antonio
1,(the tempest),ariel
2,(much ado about nothing),beatrice


In [17]:
df3 = pd.merge(result, data1, on='Speaker', how='left') ##merging the data to find the play name fot each dialogue. 
df3['Play'].fillna('miscellaneous',inplace = True) ##replacing missing values. 
df3.head(3)##output

,Speaker,Dialogue,Play
0,first citizen,,miscellaneous
1,all,"Before we proceed any further, hear me speak.",miscellaneous
2,first citizen,"Speak, speak.",miscellaneous


### Part 3

#### How impactful is the speaker on the audience?

The dialogue texts were cleaned. 
- that is punctuations were removed from text lines. 
- the lines were breaken into tokens. 
- words were lemmitized i.e., brought into their base form
- stop words were removed. 

- Sentiment analysis was conducted on the cleaned dialogues and polarity scores were added for each dialogue row. 
###### Speakers impact is figured out across two factors. the number of times he has a sentiment score and the overall polarity value he received. The positive and negative sentiments were assessed seperately. 

In [18]:
## text prerocessing  function
def process_texts(text):
  
    #Remove punctuations
    text = text.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(text)
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
     #reducing a word to its word stem 
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [19]:
result['Dialogue'] = result['Dialogue'].astype('str') ##converting object data type into string. 

In [20]:
result["processed_text"] = result['Dialogue'].apply(lambda x: process_texts(x))##applying the word cleaning method to the dialogues
print('Text Preprocessing complete.') 

Text Preprocessing complete.


#### Sentiment analysis

In [21]:
def polarity(text): ## deifining polarity using textblob(as the dataset is unlabelled)
    try:
        return TextBlob(text).sentiment ##applying textblob
    except:
        return None 
result['polarity'] = result["processed_text"].apply(polarity)
result.head(10)##output

,Speaker,Dialogue,processed_text,polarity
0,first citizen,,,"(0.0, 0.0)"
1,all,"Before we proceed any further, hear me speak.",Before proceed hear speak,"(0.0, 0.0)"
2,first citizen,"Speak, speak.",Speak speak,"(0.0, 0.0)"
3,all,You are all resolved rather to die than to fa...,You resolved rather die famish,"(0.0, 0.0)"
4,first citizen,Resolved. resolved.,Resolved resolved,"(0.0, 0.0)"
5,all,"First, you know Caius Marcius is chief enemy ...",First know Caius Marcius chief enemy people,"(0.25, 0.3333333333333333)"
6,first citizen,"We know't, we know't.",We knowt knowt,"(0.0, 0.0)"
7,all,"Let us kill him, and we'll have corn at our o...",Let u kill well corn price Ist verdict,"(0.0, 0.0)"
8,second citizen,"No more talking on't; let it be done: away, a...",No talking ont let done away away,"(0.0, 0.0)"
9,first citizen,"One word, good citizens.",One word good citizen,"(0.7, 0.6000000000000001)"


In [22]:
result[['pol','sub']] = pd.DataFrame(result["polarity"].tolist(), index= result.index)
result.head(3) ## seperating polairty score

,Speaker,Dialogue,processed_text,polarity,pol,sub
0,first citizen,,,"(0.0, 0.0)",0.0,0.0
1,all,"Before we proceed any further, hear me speak.",Before proceed hear speak,"(0.0, 0.0)",0.0,0.0
2,first citizen,"Speak, speak.",Speak speak,"(0.0, 0.0)",0.0,0.0


In [23]:
def label_sentiment (row): ##converting the polarity value into sentiment
   if row['pol'] < 0 :
      return 'Negative' #negative sentiment when polarity value <0
   if row['pol'] >= 0 :
      return 'Positive'#positive sentiment when polarity value >=0

In [24]:
result["sentiment_tb"] = result.apply (lambda row: label_sentiment(row), axis=1)
result

,Speaker,Dialogue,processed_text,polarity,pol,sub,sentiment_tb
0,first citizen,,,"(0.0, 0.0)",0.000000,0.000000,Positive
1,all,"Before we proceed any further, hear me speak.",Before proceed hear speak,"(0.0, 0.0)",0.000000,0.000000,Positive
2,first citizen,"Speak, speak.",Speak speak,"(0.0, 0.0)",0.000000,0.000000,Positive
3,all,You are all resolved rather to die than to fa...,You resolved rather die famish,"(0.0, 0.0)",0.000000,0.000000,Positive
4,first citizen,Resolved. resolved.,Resolved resolved,"(0.0, 0.0)",0.000000,0.000000,Positive
...,...,...,...,...,...,...,...
7441,sebastian,"-- And yet me thinks I see it in thy face, Wha...",And yet think see thy face What thou shouldst ...,"(0.4333333333333333, 0.7333333333333333)",0.433333,0.733333,Positive
7442,antonio,"What, art thou waking?",What art thou waking,"(0.0, 0.0)",0.000000,0.000000,Positive
7443,sebastian,Do you not hear me speak?,Do hear speak,"(0.0, 0.0)",0.000000,0.000000,Positive
7444,antonio,I do; and surely It is a sleepy language and ...,surely It sleepy language thou speakst Out thy...,"(0.11000000000000001, 0.5077777777777778)",0.110000,0.507778,Positive


#### Positive sentiment

In [25]:
result1 = result[result['sentiment_tb'] == 'Positive'] ##filtering positive sentiment dialogues

In [26]:
##impactful speaker with more positive sentiment countwise
imp_speaker_p = result1.groupby(by=['Speaker'])['sentiment_tb'].count().sort_values(ascending=False) ##grouping by speaker
imp_speaker_p = imp_speaker_p.reset_index()##setting index
imp_speaker_p.head(10)## output

,Speaker,sentiment_tb
0,gloucester,189
1,duke vincentio,156
2,menenius,145
3,coriolanus,130
4,petruchio,130
5,romeo,127
6,richard iii,116
7,of york,110
8,sicinius,107
9,isabella,106


#### As we can see, Gloucester is the dominating speaker, followed by duke vincentio ,in evoking positive sentiment to the reader,countwise. 
#### That is he spoke most no of positive dialogues among all the speakers. 

In [27]:
imp_speaker_p1 = result1.groupby(by=['Speaker'])['pol'].sum().sort_values(ascending=False) 
imp_speaker_p1 = imp_speaker_p1.reset_index()
imp_speaker_p1.head(10)

,Speaker,pol
0,gloucester,25.893746
1,duke vincentio,24.781652
2,romeo,22.037682
3,petruchio,19.502082
4,menenius,19.119244
5,juliet,18.946214
6,of york,18.146514
7,leontes,18.035352
8,sicinius,17.519861
9,queen elizabeth,16.544910


#### As we can see, Gloucester is the dominating speaker, followed by duke vincentio ,in evoking positive sentiment to the reader,even based on the overall total polarity. 
#### That is he spoke higher positively polarized statements  among all the speakers. 

#### Negative sentiment

In [28]:
result2 = result[result['sentiment_tb'] == 'Negative'] ##filtering negative sentiment dialogues

In [29]:
imp_speaker_n = result2.groupby(by=['Speaker'])['sentiment_tb'].count().sort_values(ascending=False) ##finding sentiment counts for top speakers
imp_speaker_n = imp_speaker_n.reset_index()
imp_speaker_n.head(10)

,Speaker,sentiment_tb
0,gloucester,40
1,romeo,38
2,duke vincentio,37
3,leontes,36
4,petruchio,28
5,richard ii,26
6,isabella,23
7,juliet,23
8,queen elizabeth,23
9,richard iii,22


#### As we can see, Gloucester is the dominating speaker, followed by romeo ,in evoking negative sentiment to the reader, countwise. 
#### That is he spoke most no of negative dialogues among all the speakers. 

In [30]:
imp_speaker_n1 = result2.groupby(by=['Speaker'])['pol'].sum().sort_values(ascending=False) 
imp_speaker_n1 = imp_speaker_n1.reset_index()
imp_speaker_n1.head(10)

,Speaker,pol
0,adriatic seas,-0.003125
1,clifford slain,-0.004015
2,english crown,-0.004463
3,dion,-0.012500
4,depart again,-0.014231
5,volsce,-0.014286
6,lord stanley,-0.021212
7,francisco,-0.025000
8,corioli,-0.025000
9,rome gates,-0.028125


#### As we can see, adratic seas is the dominating speake ,in evoking negative sentiment to the reader,based on the overall total polarity. 
#### That is he spoke higher negatively polarized statements  among all the speakers. 

#### Hence, overall Gloucester appears to be the most impactful and important speaker in both positive sentiment and negative sentimental dialogues. 